<a href="https://colab.research.google.com/github/Adrok24/Proyecto-Integrador/blob/branch_1/Preprocessing_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importaciones y descargas

In [117]:
from google.colab import files
from google.colab import drive
import os
from os.path import isfile, join
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
pd.options.display.max_colwidth = 150

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [118]:
!pip install nltk

In [119]:
!unrar x '/content/drive/My Drive/Colab Notebooks/desafio final/txt_books_spa.rar'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/Colab Notebooks/desafio final/txt_books_spa.rar


Would you like to replace the existing file txt_books_spa/txt_books_spa/cortazar txt/alvo_el_crepusculo_-_Julio_Cortazar.txt
198858 bytes, modified on 2021-02-02 23:32
with a new one
198858 bytes, modified on 2021-02-02 23:32

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit q

Program aborted


In [126]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Generar una lista de nombres

In [121]:
folder = '/content/drive/My Drive/Colab Notebooks/desafio final/'

path_to_names = os.path.join(folder, 'list_names.csv')
data_names = pd.read_csv(path_to_names)

In [150]:
list_names = data_names.Name.drop_duplicates().tolist()

In [151]:
delete_from_Names = ['Donde','Por','Para','Pero', 'Al','Sin','Con','Por', 'De','Desde','Por','Para','Hasta','Contra','El','Los','Las','Sus','Esas','Esos','Todos','Mis','Tus', 'Ese', 'Aquel', 'Esos', 'Ellos', 'En', 'Me', 'Te', 'Se', 'Le', 'Ve']

In [152]:
len(list_names)

43675

In [154]:
for x in delete_from_Names:
  if (x in list_names):
    list_names.remove(x)

len(list_names)

43663

# Generacion de un listado de libros

In [155]:
mypath = '/content/txt_books_spa' 

# list_files = [join(root, name_file) for root, dirs, name_file in os.walk(mypath, topdown=False)] 

count_files = 0
dict_paths = {}
for root, dirs, files in os.walk(mypath, topdown=False):
    for name_file in files:
        dict_paths[count_files] = {"path": os.path.join(root, name_file), "name": name_file.replace('.txt', ''), "genre": root.split('/')[4].split(' ')[0], "category" : "Book"}
        count_files = count_files + 1

df_path = pd.DataFrame.from_dict(dict_paths, "index")
display(df_path.shape)

(154, 4)

# Separacion en lineas

In [156]:
def get_lines(line, book, dict_book_final, count_lines):
  if (len(line) > 130):
    split_line = line.split('.')
    if (len(split_line) == 1):
      split_line = line.split(',')
    for s_line in split_line:
      if ((len(s_line) < 130) and (len(s_line) > 30)):
        dict_book_final[count_lines] = {"line": s_line, "len": len(s_line), "book": book['name'], "genre": book['genre'], "category": book['category']}
        count_lines = count_lines + 1
  elif (len(line) > 30):  
    dict_book_final[count_lines] = {"line": line, "len": len(line), "book": book['name'], "genre": book['genre'], "category": book['category']}
    count_lines = count_lines + 1
  return dict_book_final, count_lines

In [157]:
dict_book_final = {}
count_lines = 0
for index, book in df_path.iterrows():
  corpus = open(book['path'], "r")
  cadena = corpus.read()
  list_cadena = sent_tokenize(cadena, language='spanish')
  print('cant sent:',len(list_cadena),book['name'])

  for line in list_cadena:
    if (line.find('\n') == -1):
      dict_book_final, count_lines = get_lines(line, book, dict_book_final, count_lines)
    else:
      split_line = line.split('\n')
      for s_line in split_line:
        dict_book_final, count_lines = get_lines(s_line, book, dict_book_final, count_lines)

cant sent: 1768 Bestiario_-_Julio_Cortazar
cant sent: 950 mbradas_culturales_-_Julio_Cortazar
cant sent: 788 Un_tal_Lucas_-_Julio_Cortazar
cant sent: 910 Obra_critica_I_-_Julio_Cortazar
cant sent: 11572 Cartas_1937-1954_-_Julio_Cortazar
cant sent: 8700 Los-premios-Julio-Cortazar
cant sent: 2679 ases_de_literatura_-_Julio_Cortazar
cant sent: 2328 Juegos-Julio-Cortazar
cant sent: 12746 Cartas_1955-1964_-_Julio_Cortazar
cant sent: 2711 Los-autonautas-de-la-cosmopista-Julio-Cortazar
cant sent: 162 Silvalandia_-_Julio_Cortazar
cant sent: 3665 Obra_critica_II_-_Julio_Cortazar
cant sent: 1187 Deshoras_-_Julio_Cortazar
cant sent: 2213 on_y_otros_relatos_-_Julio_Cortazar
cant sent: 1472 Nicaragua-tan-violentamente-dulce-Julio-Cortazar
cant sent: 1185 alvo_el_crepusculo_-_Julio_Cortazar
cant sent: 2322 Las-armas-secretas-Julio-Cortazar
cant sent: 1057 mos_tanto_a_Glenda_-_Julio_Cortazar
cant sent: 3042 Pasajes_-_Julio_Cortazar
cant sent: 4446 or_y_otros_relatos_-_Julio_Cortazar
cant sent: 2161 O

In [158]:
df_book_lines = pd.DataFrame.from_dict(dict_book_final, "index")
display(df_book_lines)

,line,len,book,genre,category
0,Bestiario es el primer libro de relatos que Julio Cortázar publica con su auténtico nombre.,91,Bestiario_-_Julio_Cortazar,cortazar,Book
1,No hay en estas ocho obras maestras ni el menor balbuceo ni resacas juveniles: son perfectas.,93,Bestiario_-_Julio_Cortazar,cortazar,Book
2,"Sorpresa o incomodidad son, en cada texto, un condimento que se agrega al placer indescriptible de su lectura.",110,Bestiario_-_Julio_Cortazar,cortazar,Book
3,Retoque de cubierta: Un_Tal_Lucas,33,Bestiario_-_Julio_Cortazar,cortazar,Book
4,"A Paco, que gustaba de mis relatos",34,Bestiario_-_Julio_Cortazar,cortazar,Book
...,...,...,...,...,...
348755,"Esa dirección ya formaba parte del pasado, de otra vida que ya no existía.",74,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348756,Ahora le pertenecía a otra familia que la querría tanto como ella.,66,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348757,"El 44 de Charles Street era un episodio de su vida, mucho más que una simple casa.",82,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348758,Aquel capítulo ya estaba cerrado.,33,arles_street_no_44_-_Danielle_Steel,Novelas,Book


# Filtrado y preprocesamiento

In [159]:
def preprocess_sentence(sentence):
  sentence = sentence.strip()
  # vamos a crear espacios entre las frases y los puntos:
  # ej: "he is a boy." => "he is a boy ."
  sentence = re.sub(r"([?.!,:])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  sentence = re.sub(r'[A-Z]{2,20}', " ", sentence)
  sentence = re.sub(r'\d+', "@number", sentence) 
  sentence = sentence.strip()
  return sentence

In [160]:
df_book_lines.iloc[:,0] = df_book_lines.iloc[:,0].apply(preprocess_sentence)

In [161]:
df_book_lines

,line,len,book,genre,category
0,Bestiario es el primer libro de relatos que Julio Cortázar publica con su auténtico nombre .,91,Bestiario_-_Julio_Cortazar,cortazar,Book
1,No hay en estas ocho obras maestras ni el menor balbuceo ni resacas juveniles : son perfectas .,93,Bestiario_-_Julio_Cortazar,cortazar,Book
2,"Sorpresa o incomodidad son , en cada texto , un condimento que se agrega al placer indescriptible de su lectura .",110,Bestiario_-_Julio_Cortazar,cortazar,Book
3,Retoque de cubierta : Un_Tal_Lucas,33,Bestiario_-_Julio_Cortazar,cortazar,Book
4,"A Paco , que gustaba de mis relatos",34,Bestiario_-_Julio_Cortazar,cortazar,Book
...,...,...,...,...,...
348755,"Esa dirección ya formaba parte del pasado , de otra vida que ya no existía .",74,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348756,Ahora le pertenecía a otra familia que la querría tanto como ella .,66,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348757,"El @number de Charles Street era un episodio de su vida , mucho más que una simple casa .",82,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348758,Aquel capítulo ya estaba cerrado .,33,arles_street_no_44_-_Danielle_Steel,Novelas,Book


In [162]:
# Enmascaramos los nombres propios, y marcamos algunas lineas que tienen caracteres especiales para luego filtrarlas

from IPython.display import clear_output
list_new_rows = [] 
for index, row in df_book_lines.iterrows():
  new_row = row['line']
  for word in row['line'].split(' '):
    if word in list_names:
      new_row = row['line'].replace(word, '@name')
    if word in [':','[',']','_']:
      new_row = row['line'] = '[DELETE]'

  clear_output(wait=True)
  print(index)   
  list_new_rows.append(new_row)

348759


In [163]:
df_book_lines.iloc[:,0] = list_new_rows

In [164]:
df_book_lines

,line,len,book,genre,category
0,Bestiario es el primer libro de relatos que @name Cortázar publica con su auténtico nombre .,91,Bestiario_-_Julio_Cortazar,cortazar,Book
1,[DELETE],93,Bestiario_-_Julio_Cortazar,cortazar,Book
2,"Sorpresa o incomodidad son , en cada texto , un condimento que se agrega al placer indescriptible de su lectura .",110,Bestiario_-_Julio_Cortazar,cortazar,Book
3,[DELETE],33,Bestiario_-_Julio_Cortazar,cortazar,Book
4,"A @name , que gustaba de mis relatos",34,Bestiario_-_Julio_Cortazar,cortazar,Book
...,...,...,...,...,...
348755,"@name dirección ya formaba parte del pasado , de otra vida que ya no existía .",74,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348756,Ahora le pertenecía a otra familia que la querría tanto como ella .,66,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348757,"El @number de @name Street era un episodio de su vida , mucho más que una simple casa .",82,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348758,Aquel capítulo ya estaba cerrado .,33,arles_street_no_44_-_Danielle_Steel,Novelas,Book


In [165]:
# Obsrevamos lineas repetidas
df_book_lines.line.value_counts().head(12)

[DELETE]                                                                                              10653
                                                                                                         51
De Canciones de Mihyar el de Damasco @number                                                             21
Versión de @name Martínez Montávez                                                                       21
A    È  Y  ÍA                                                                                            18
ÓN                                                                                                       10
Y                                                                                                        10
Es una de las autoras más conocidas y leídas en el mundo entero .                                        10
De sus novelas , traducidas a veintiocho idiomas , se han vendido @number millones de ejemplares .       10
En @number se le otorgó la c

In [166]:
#Creamos una mascara para el borrado de 

borrar = df_book_lines.line.value_counts().head(12).index.tolist()
mask_del = df_book_lines.line.apply(lambda x: True if x in borrar else False) 
df_book_lines[mask_del]

,line,len,book,genre,category
1,[DELETE],93,Bestiario_-_Julio_Cortazar,cortazar,Book
3,[DELETE],33,Bestiario_-_Julio_Cortazar,cortazar,Book
28,[DELETE],102,Bestiario_-_Julio_Cortazar,cortazar,Book
41,[DELETE],121,Bestiario_-_Julio_Cortazar,cortazar,Book
91,[DELETE],104,Bestiario_-_Julio_Cortazar,cortazar,Book
...,...,...,...,...,...
347610,[DELETE],35,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348297,[DELETE],105,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348503,[DELETE],123,arles_street_no_44_-_Danielle_Steel,Novelas,Book
348677,[DELETE],88,arles_street_no_44_-_Danielle_Steel,Novelas,Book


In [167]:
# Ejecutamos el Drop
df_book_lines.drop(df_book_lines[mask_del].index, inplace=True)

In [168]:
# Evaluamos el shape actual
df_book_lines.drop_duplicates(inplace=True)
df_book_lines.shape[0]

337358

In [212]:
def remove_special_chars(sentence):
  sentence = sentence.lower().strip()
  accentedCharacters = "áéíóúÁÉÍÓÚñÑäëïöüÜ"
  sentence = re.sub(r"[^a-zA-Z" + accentedCharacters + "@?¿¡.!,]+", " ", sentence)
  sentence = re.sub(r"([?¿¡.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  sentence = sentence.strip('.')
  sentence = sentence.strip(',')
  return sentence

In [213]:
df_book_lines.iloc[:,0] = df_book_lines.iloc[:,0].apply(remove_special_chars)

In [202]:
# Regeneramos la longitud en caracteres post procesamiento
df_book_lines.len = df_book_lines.line.apply(lambda x : len(x))

In [203]:
# Creamos una mascara para eliminar lineas con menos de 35 caracteres

mask_largo = df_book_lines.len < 35
print(mask_largo.sum())

893


In [204]:
# Ejecutamos el Drop
# Evaluamos el shape actual
df_book_lines.drop(df_book_lines[mask_largo].index, inplace=True)
df_book_lines.shape[0]

316618

In [214]:
# Obtenemos algunos samples para evaluar el resultado de nuestro corpus

df_book_lines.sample(15)

,line,len,book,genre,category
74822,es la mano que toma todo lo que le pertenece en mi cuerpo,59,_frente_al_espejo_-_Denis_Hernandez,poesía,Book
345043,"el pequeño asintió , así que @name llenó un bol de cereales , los cubrió de leche y le dio un plato para el plátano",121,arles_street_no_44_-_Danielle_Steel,Novelas,Book
155763,gordon querrá que vuelva a parís en cuanto me recupere,56,El_beso_-_Danielle_Steel,Novelas,Book
88593,en tibias caras de espantados ojos,36,_grandes_mujeres_-_Alfonsina_Storni,poesía,Book
34751,"mamá no ganaba para sustos , entre el chico de la modista que se había quemado el brazo con leche hirviendo y el boby sarnoso",129,Las-armas-secretas-Julio-Cortazar,cortazar,Book
50014,"tener opiniones es inevitable , normal tener convicciones lo es menos",73,ente_de_haber_nacido_-_E._M._Cioran,filosofia,Book
187592,"para aquellos que jamás conocieron esas cosas , esto tiene el poder de una fuerza hipnótica",95,El_anillo_-_Danielle_Steel,Novelas,Book
159762,"ahora , mis padres ya no serán un problema",46,Guarida_de_condenados_-_A._Rolcest,Novelas,Book
283129,pero un cruel golpe del destino había acabado con todo,56,El_rancho_-_Danielle_Steel,Novelas,Book
27508,"@number h . @number c al sol , lo que no nos ha impedido explorar , como intrépidos expedicionarios que somos , el arqueódromo",136,Los-autonautas-de-la-cosmopista-Julio-Cortazar,cortazar,Book


In [215]:
# Guardamos el resultado
df_book_lines.to_csv('/content/drive/My Drive/Colab Notebooks/desafio final/train_corpus_v2.csv')

In [148]:
# Generamos una funcion para buscar palabras extranjeras, nos sirve para evaluar pusibles errores
def find_word(palabra):
  for index, row in df_book_lines.iterrows():
    new_row = row['line']
    for word in row['line'].split(' '):
      if word in [palabra]:
        print(row['book'])
  

In [149]:
find_word('passagem')

Poemas_-_Fernando_Pessoa
Poemas_-_Fernando_Pessoa
Poemas_-_Fernando_Pessoa


In [92]:
# mask_clean_book = df_book_lines.book == 'Poemas_-_Fernando_Pessoa'
# mask_clean_book.sum()

# print(df_book_lines.shape[0])
# df_book_lines.drop(df_book_lines[mask_clean_book].index, inplace=True)
# print(df_book_lines.shape[0])